In [6]:
from datasets import load_dataset, DatasetDict, ClassLabel, Dataset
from src.utils import map_category

aux_data = load_dataset("real-jiakai/arxiver-with-category")


In [7]:
aux_data.set_format(type="pandas")
aux_df = aux_data["train"][:]
aux_df["label"] = aux_df["primary_category"].apply(map_category)
aux_df["title"] = aux_df["title"].str.replace("\n  ", " ")
aux_df["text"] = aux_df["title"] + "\n" + aux_df["abstract"]
aux_df = aux_df[["text", "label"]]


In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    aux_df, 
    test_size=0.1,
    stratify=aux_df["label"],
    random_state=42
)

cpt_data = DatasetDict({
    "train": Dataset.from_pandas(train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False)
})

labels = sorted(train_df["label"].unique())
class_label = ClassLabel(names=labels)

cpt_data = cpt_data.cast_column("label", class_label)

cpt_data.save_to_disk("data/processed/cpt_data")


Casting the dataset:   0%|          | 0/57021 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6336 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/57021 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6336 [00:00<?, ? examples/s]

In [9]:
from transformers import DistilBertForMaskedLM, DistilBertTokenizer, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", 
                    truncation=True, max_length=512)

tokenized_cpt_data = cpt_data.map(tokenize, batched=True, remove_columns=["label"])


Map:   0%|          | 0/57021 [00:00<?, ? examples/s]

Map:   0%|          | 0/6336 [00:00<?, ? examples/s]

In [10]:
# Data collator for masked language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="distilbert-base-uncased-cpt-arxiv",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,
    fp16=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=1000,
    logging_dir='logs',
    logging_steps=500,              
    dataloader_num_workers=4,       
    eval_strategy="steps",   
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cpt_data["train"],
    eval_dataset=tokenized_cpt_data["validation"],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Run continued pretraining
trainer.train()


Step,Training Loss,Validation Loss
500,2.379900,2.087670
1000,2.141800,1.982328
1500,2.057600,1.896975
2000,1.995500,1.849949
2500,1.946000,1.822771
3000,1.918900,1.789547
3500,1.895100,1.776106
4000,1.862400,1.759627
4500,1.849200,1.749570
5000,1.836900,1.724996


There were missing keys in the checkpoint model loaded: ['vocab_projector.weight'].


TrainOutput(global_step=12500, training_loss=1.846195576171875, metrics={'train_runtime': 2370.3894, 'train_samples_per_second': 240.555, 'train_steps_per_second': 7.518, 'total_flos': 5.302167865412198e+16, 'train_loss': 1.846195576171875, 'epoch': 7.014590347923681})

In [11]:
# Save domain-adapted model
model.save_pretrained("distilbert-arxiv-domain-adapted")
tokenizer.save_pretrained("distilbert-arxiv-domain-adapted")


('distilbert-arxiv-domain-adapted/tokenizer_config.json',
 'distilbert-arxiv-domain-adapted/special_tokens_map.json',
 'distilbert-arxiv-domain-adapted/vocab.txt',
 'distilbert-arxiv-domain-adapted/added_tokens.json')